##Imports

In [1]:
import re
import pandas as pd
import json
import numpy as np
import seaborn as sns
import nltk
from bertopic import BERTopic

/home/lucas/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Extração de Dados

In [2]:
def df_tweets_candidatos(json_filename):
    df = pd.read_json(json_filename).drop(columns = ['replies'])
    return df


def df_tweets_respostas(json_filename):
    df = pd.read_json(json_filename).dropna().reset_index()[['replies']]
    ds = []
    for replies in df['replies'].to_list():
        for reply in replies:
            ds.append(reply)

    reply_df = pd.DataFrame (ds).drop(columns = ['attachments'])
    return reply_df

def df_tweets_cadidatos_respostas(json_filename):
    df_tweets_cand = df_tweets_candidatos(json_filename)
    df_tweets_reps = df_tweets_respostas(json_filename)

    return (df_tweets_cand, df_tweets_reps)

# Pré-Processamento

In [3]:
tweets = df_tweets_candidatos('./datasets/dataset.json')


# Adicionar nomes dos candidatos no dataframe
for index, row in tweets.iterrows():
    candidato = ''
    if row.author_id == 2670726740:
      candidato = 'lula'
    elif row.author_id == 128372940:
      candidato = 'bolsonaro'
    elif row.author_id == 33374761:
      candidato = 'ciro'
    else:
      candidato = 'n/d'
    tweets.at[index, 'candidato'] = candidato

print(f'Número de tweets: {len(tweets)}\n')
print(tweets.head())

Número de tweets: 300

    author_id      conversation_id                created_at  \
0  2670726740  1560319888643719168 2022-08-18 17:37:05+00:00   
1  2670726740  1560299315905515520 2022-08-18 16:15:20+00:00   
2  2670726740  1560285152282034176 2022-08-18 15:19:03+00:00   
3  2670726740  1560266077329694720 2022-08-18 14:03:15+00:00   
4  2670726740  1560248955753095168 2022-08-18 12:55:13+00:00   

                                                text                   id  \
0  As pessoas pensam que o sucesso dos nossos gov...  1560319888643719168   
1  Já em Belo Horizonte para o primeiro comício d...  1560299315905515520   
2  Ontem Lula recebeu a visita de representantes ...  1560285152282034176   
3  RT @verdadenarede: Damares condenada! No final...  1560266077329694720   
4                      Nove. https://t.co/hfMesT3LHq  1560248955753095168   

  candidato  
0      lula  
1      lula  
2      lula  
3      lula  
4      lula  


In [4]:
nltk.download('stopwords')
pt_stop = set(nltk.corpus.stopwords.words('portuguese'))

[nltk_data] Downloading package stopwords to /home/lucas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import re
from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

def preprocess_text(document):

        # removing urls
        document = re.sub(r'http\S+', '', document)

        # removing mentions
        document = re.sub(r'\B@\w+', '', document)

        # remove all the special characters
        document = re.sub(r'\W', ' ', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # converting to lowercase
        document = document.lower()

        # lemmatization
        tokens = document.split()
        tokens = [stemmer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in pt_stop]
        tokens = [word for word in tokens if len(word) > 3]

        return ' '.join(tokens)

# Modelagem de Tópicos

In [6]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to /home/lucas/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/lucas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# create model 
model = BERTopic(language="multilingual", verbose=True, min_topic_size=3, top_n_words=10, calculate_probabilities=True)

# convert to list 
docs = tweets.text.to_list()

# Pre-processamento
for i, doc in enumerate(docs):
    docs[i] = preprocess_text(doc)

tweets_topic, probabilities = model.fit_transform(docs)

Batches: 100%|██████████| 10/10 [00:07<00:00,  1.34it/s]
2022-08-27 22:46:00,560 - BERTopic - Transformed documents to Embeddings
2022-08-27 22:46:10,660 - BERTopic - Reduced dimensionality
2022-08-27 22:46:11,228 - BERTopic - Clustered reduced embeddings


Adicionando o tópico de cada tweet no Dataframe

In [8]:
topic_names = model.generate_topic_labels(nr_words=4)

# Array indicando o tópico de cada tweet (numérico -> índice do array "topic_names" + 1)
# print(tweets_topic)
# Array com todos os tópicos nomeados
# print(topic_names)

# print(docs)


for index, tweet_topic in enumerate(tweets_topic):
    tweets.at[index, 'topico'] = topic_names[tweet_topic+1]
    tweets.at[index, 'processed_text'] = docs[index]

# DEBUG
# display(tweets)
# tweets.to_csv('/home/rodrigo98rm/Documents/ufabc/pln/projeto-2/pln-2022.2/src/output/dataframe.csv', sep='\t', encoding='utf-8')

Abaixo, os tópicos identificados entre todos os tweets

In [9]:
freq = model.get_topic_info()
print(freq)

    Topic  Count                                         Name
0      -1     51                    -1_ciro_lula_brasil_todos
1       0     28               0_democracia_carta_deus_drogas
2       1     25                 1_luladay_noite_gettr_baixar
3       2     24  2_esperança_melhor_brasildaesperança_brasil
4       3     22      3_milhões_brasileiros_famílias_economia
5       4     17              4_pequenos_deixar_custou_quanto
6       5     13        5_país_brasildaesperança_brasil_fazer
7       6     13         6_redução_combustíveis_energia_preço
8       7     12               7_brasil_jovens_setor_empregos
9       8     10                    8_eleição_voto_falar_café
10      9      8               9_campanha_mudar_oficial_redes
11     10      8    10_paixões_ódios_despolitizadas_reeleição
12     11      7          11_programa_renda_mínima_distribuir
13     12      7                12_todos_amanhã_10h30_jéssica
14     13      6          13_corrupção_chamar_errado_história
15     1

# Análise de Sentimentos

# Similaridade de Textos

Imports necessários

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Funções utilizadas para o cálculo de similaridade

In [11]:

#Gera lista em que cada índice contém um documento relativo a cada candidato por tópico escolhido.
def monta_docs_concat_topico(df, topico, coluna_alvo, candidatos):
    docs = []
    df_topico = df[(df['topico'] == topico)]
    for cand in candidatos:
        df_topico_cand = df_topico[df_topico['candidato'] == cand]
        texto = ' '.join(df_topico_cand[coluna_alvo].to_list())
        docs.append(texto)
    return docs

#Gera lista em que cada índice contem um documento com todos os tweets de cada candidato separados por espaço.
def monta_docs_concat_candidato(df, coluna_alvo, candidatos):
    docs = []
    for cand in candidatos:
        df_cand = df[df['candidato'] == cand]
        texto = ' '.join(df_cand[coluna_alvo].to_list())
        docs.append(texto)
    return docs

stop_words_ptbr = nltk.corpus.stopwords.words('portuguese')

#A função normaliza_tweet não foi necessária, decidimos reutilizar o pré processamento já realizado na modelagem de tópicos
def normaliza_tweet(tweet):
    tweet = re.sub(r'[^a-zA-Z0-9\s]', '', tweet, re.I|re.A)
    tweet = tweet.lower()
    tweet = tweet.strip()
    tokens = nltk.word_tokenize(tweet)
    tokens_filtrados = [token for token in tokens if token not in stop_words_ptbr]
    tweet_tokens = ' '.join(tokens_filtrados)
    return tweet_tokens

#Gerando matriz TF-IDF
def gera_matriz_tfidf(corpus_vetorizado):
    tf = TfidfVectorizer(ngram_range=(1, 2), min_df=1)
    matriz_tfidf = tf.fit_transform(corpus_vetorizado)
    return matriz_tfidf

#Gera similaridades após ter as representações dos documentos em TF-IDF
def similaridade_pares(matriz_tfidf):
    dict_docs_sim = cosine_similarity(matriz_tfidf)
    df_sim = pd.DataFrame(dict_docs_sim)
    return df_sim

#Renomeia colunas para melhor entendimento de quem há maior similirdade com quem
def renomear_colunas_linhas(df, candidatos):
    df.columns = candidatos
    df.index = candidatos
    return df

Com isso, pensando em calcular a similaridade entre os candidados considerando todos os tweets e tweets de tópicos específicos, criamos as seguintes funções:

In [12]:
def calcula_similaridade_candidatos(df, coluna_alvo, candidatos):
    lista_docs = monta_docs_concat_candidato(df, coluna_alvo, candidatos)
    corpus_normalizado = lista_docs
    matriz_tfidf = gera_matriz_tfidf(corpus_normalizado)
    df_sim = similaridade_pares(matriz_tfidf)
    df_sim_cands = renomear_colunas_linhas(df_sim, candidatos)

    return df_sim

#Calcula similaridade entre os candidatos dado um tópico
def calcula_similaridade_topico(df, topico, coluna_alvo, candidatos):
    lista_docs = monta_docs_concat_topico(df, topico, coluna_alvo, candidatos)
    corpus_normalizado = lista_docs
    matriz_tfidf = gera_matriz_tfidf(corpus_normalizado)
    df_sim = similaridade_pares(matriz_tfidf)
    df_sim_cands = renomear_colunas_linhas(df_sim, candidatos)

    return df_sim_cands

def calcula_similaridade_topicos(df, topicos, coluna_alvo, candidatos):
    dfs_sim = []
    for topico in topicos:
        dfs_sim.append(calcula_similaridade_topico(df, topico, coluna_alvo, candidatos))
        
    sim_dict = dict(zip(topicos, dfs_sim))
    return sim_dict

In [13]:
df_freq = model.get_topic_info()
topicos_mais_frequentes = df_freq['Name'].to_list()[0:4]
topicos_mais_frequentes

['-1_ciro_lula_brasil_todos',
 '0_democracia_carta_deus_drogas',
 '1_luladay_noite_gettr_baixar',
 '2_esperança_melhor_brasildaesperança_brasil']

**Comparando Similaridade entre os candidatos para os tópicos mais frequentes e utilizando o texto processado já na fase de modelagem**

In [14]:
candidatos = ["lula", "bolsonaro", "ciro"]
analise_similaridade = calcula_similaridade_topicos(tweets, topicos_mais_frequentes, "processed_text", candidatos)
for topico in analise_similaridade.keys():
    print(f"Para tópico = {topico}")
    print(analise_similaridade[topico])
    print('----------------------------------')

Para tópico = -1_ciro_lula_brasil_todos
               lula  bolsonaro      ciro
lula       1.000000   0.037415  0.034253
bolsonaro  0.037415   1.000000  0.013645
ciro       0.034253   0.013645  1.000000
----------------------------------
Para tópico = 0_democracia_carta_deus_drogas
               lula  bolsonaro      ciro
lula       1.000000   0.051609  0.021486
bolsonaro  0.051609   1.000000  0.044637
ciro       0.021486   0.044637  1.000000
----------------------------------
Para tópico = 1_luladay_noite_gettr_baixar
               lula  bolsonaro  ciro
lula       1.000000   0.029581   0.0
bolsonaro  0.029581   1.000000   0.0
ciro       0.000000   0.000000   1.0
----------------------------------
Para tópico = 2_esperança_melhor_brasildaesperança_brasil
               lula  bolsonaro     ciro
lula       1.000000   0.015454  0.06464
bolsonaro  0.015454   1.000000  0.03052
ciro       0.064640   0.030520  1.00000
----------------------------------


**Comparando Similaridade entre os candidatos entre todos os tweets (ou seja, todos os tópicos) e utilizando o texto processado já na fase de modelagem**

In [15]:
analise_sim_cand = calcula_similaridade_candidatos(tweets, "processed_text", candidatos)
analise_sim_cand

,lula,bolsonaro,ciro
lula,1.000000,0.147990,0.209275
bolsonaro,0.147990,1.000000,0.115364
ciro,0.209275,0.115364,1.000000


**O quão similar é o discurso entre os candidatos para os tópicos encontrados?**

Aqui, em nossa primeira tentativa decidimos usar diretamente os **textos normalizados** e obtivemos uma baixa similaridade entre os candidatos, tanto no escopo por tópico quanto entre todos os tópicos, o que foi de acordo com nossa hipótese, dado que são 3 candidatos com percepções bem distintas entre os 3, então tivemos um resultado bem satisfatório, obtendo similaridades menores que 10% entre os candidatos para cada um dos tópicos, e obtendo similaridades menores que 21% entre os candidatos ao olhar todos os tópicos.

Além disso, a maior similaridade se dá entre Lula e Ciro, que também era esperado por nosso grupo dada o discurso praticamente antagônico entre Lula e Bolsonaro.

**Comparando Similaridade entre os candidatos para os tópicos mais frequentes e utilizando o texto não-processado já na fase de modelagem**

In [16]:
candidatos = ["lula", "bolsonaro", "ciro"]
analise_similaridade = calcula_similaridade_topicos(tweets, topicos_mais_frequentes, "text", candidatos)
for topico in analise_similaridade.keys():
    print(f"Para tópico = {topico}")
    print(analise_similaridade[topico])
    print('----------------------------------')

Para tópico = -1_ciro_lula_brasil_todos
               lula  bolsonaro      ciro
lula       1.000000   0.358968  0.247977
bolsonaro  0.358968   1.000000  0.226458
ciro       0.247977   0.226458  1.000000
----------------------------------
Para tópico = 0_democracia_carta_deus_drogas
               lula  bolsonaro      ciro
lula       1.000000   0.185648  0.124881
bolsonaro  0.185648   1.000000  0.313937
ciro       0.124881   0.313937  1.000000
----------------------------------
Para tópico = 1_luladay_noite_gettr_baixar
               lula  bolsonaro      ciro
lula       1.000000   0.539863  0.193510
bolsonaro  0.539863   1.000000  0.256996
ciro       0.193510   0.256996  1.000000
----------------------------------
Para tópico = 2_esperança_melhor_brasildaesperança_brasil
               lula  bolsonaro      ciro
lula       1.000000   0.132421  0.163316
bolsonaro  0.132421   1.000000  0.146598
ciro       0.163316   0.146598  1.000000
----------------------------------


**Comparando Similaridade entre os candidatos entre todos os tweets (ou seja, todos os tópicos) e utilizando o texto não-processado já na fase de modelagem**

In [17]:
analise_sim_cand = calcula_similaridade_candidatos(tweets, "text", candidatos)
analise_sim_cand

,lula,bolsonaro,ciro
lula,1.000000,0.636385,0.647811
bolsonaro,0.636385,1.000000,0.666628
ciro,0.647811,0.666628,1.000000


**O quão similar é o discurso entre os candidatos no geral?**

Nesta segunda tentativa tentamos utilizar o texto não normalizado e tivemos similaridades maiores em todos os contextos testados, sendo isso provavelmente ocasionado pela ocorrência de stop-words que estejam enviesando nosso resultado (visto que elas aparecem em grande em um número em um texto). No entanto, é possível perceber que a relação de maiores e menores similaridades entre os candidates persistem mesmo para a versão não normalizada.